<a href="https://colab.research.google.com/github/ArkS0001/FLIPKART-GRID-6.0/blob/main/Try1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install pycryptodome pqcrypto


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 7.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 34.9 MB/s eta 0:00:00
  Created wheel for pqcrypto: filename=pqcrypto-0.1.3-cp310-cp310-manylinux_2_35_x86_64.whl size=2119074 sha256=2b672a6858bc7d2a88e0495f25ddabd1a06dafaf15926ae0d48909e292899a9b
  Stored in directory: /root/.cache/pip/wheels/89/13/4f/b2c678a9f6c473acb0d259d47c30387bab4f836f28dab19f1c
Successfully built pqcrypto


Encryption and Decryption using Kyber

In [ ]:
from pqcrypto.kem.kyber512 import generate_keypair, encrypt, decrypt

# Key Generation
public_key, secret_key = generate_keypair()

# Encryption on the client side (data to send to server)
message = b"Sensitive API data"
ciphertext, shared_secret_enc = encrypt(message, public_key)

# Decryption on the server side
decrypted_message, shared_secret_dec = decrypt(ciphertext, secret_key)

print(f"Original Message: {message}")
print(f"Decrypted Message: {decrypted_message}")


ModuleNotFoundError: No module named 'pqcrypto._kem.kyber512'

Authentication using Dilithium

In [ ]:
from pqcrypto.sign.dilithium2 import generate_keypair, sign, verify

# Generate public/private keypair for authentication
public_key, secret_key = generate_keypair()

# Sign a message on the client side (message authentication)
message = b"API request data"
signature = sign(message, secret_key)

# Verify signature on the server side
try:
    verified_message = verify(signature, public_key)
    print("Message verified:", verified_message == message)
except Exception as e:
    print("Verification failed:", e)


Integrating in API

In [ ]:
from flask import Flask, request, jsonify
from pqcrypto.kem.kyber512 import generate_keypair, encrypt, decrypt
from pqcrypto.sign.dilithium2 import generate_keypair as gen_auth_keypair, sign, verify

app = Flask(__name__)

# Kyber keypair for encryption
kyber_pub_key, kyber_priv_key = generate_keypair()

# Dilithium keypair for signing
dilithium_pub_key, dilithium_priv_key = gen_auth_keypair()

@app.route('/secure-endpoint', methods=['POST'])
def secure_endpoint():
    # Data received from the client (assumed to be encrypted)
    encrypted_data = request.json['encrypted_data']
    signature = request.json['signature']

    # Decrypt data
    try:
        decrypted_data, _ = decrypt(bytes.fromhex(encrypted_data), kyber_priv_key)
    except Exception as e:
        return jsonify({"error": "Decryption failed"}), 400

    # Verify signature
    try:
        verified_message = verify(bytes.fromhex(signature), dilithium_pub_key)
        if verified_message != decrypted_data:
            return jsonify({"error": "Verification failed"}), 400
    except Exception as e:
        return jsonify({"error": "Verification exception"}), 400

    return jsonify({"message": "Data secure and authenticated"}), 200

if __name__ == '__main__':
    app.run(debug=True)


Workflow:

    Client Side:
        Encrypt the API request data using Kyber’s public key.
        Sign the encrypted data using Dilithium’s private key.
        Send the encrypted data and signature to the server.

    Server Side:
        Decrypt the encrypted data using Kyber’s private key.
        Verify the signature using Dilithium’s public key to ensure data integrity and authenticity.

Example Client Request

In [ ]:
import requests
from pqcrypto.kem.kyber512 import encrypt
from pqcrypto.sign.dilithium2 import sign

message = b"Sensitive API request data"

# Encrypt with server's public key
ciphertext, _ = encrypt(message, kyber_pub_key)

# Sign the message
signature = sign(ciphertext, dilithium_priv_key)

# Send to server
response = requests.post(
    'http://localhost:5000/secure-endpoint',
    json={
        'encrypted_data': ciphertext.hex(),
        'signature': signature.hex(),
    }
)

print(response.json())
